In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import math

In [2]:
data = pd.read_csv("OJ.csv")

In [3]:
x = data.drop(columns = ['Purchase', 'Unnamed: 0'])
y = data['Purchase']
y.head()

0    CH
1    CH
2    CH
3    MM
4    CH
Name: Purchase, dtype: object

In [4]:
x["Store7"] = x["Store7"].astype('category')
x["Store7"] = x["Store7"].cat.codes
x.head()

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,0,0.000000,0.000000,0.24,1
1,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,0,0.150754,0.000000,0.24,1
2,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,0,0.000000,0.091398,0.23,1
3,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,0,0.000000,0.000000,0.00,1
4,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,1,0.000000,0.000000,0.00,0


In [5]:
test_frac = (len(data)-800.0)/len(data)
test_frac

0.2523364485981308

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_frac)

In [7]:
print(len(X_train),len(X_test))

800 270


##### a)

In [8]:
y_train.value_counts('CH')

CH    0.61
MM    0.39
Name: Purchase, dtype: float64

In [9]:
y_test.value_counts('CH')

CH    0.611111
MM    0.388889
Name: Purchase, dtype: float64

In [10]:
X_test.head()

,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
77,251,4,1.99,2.23,0.0,0.00,0,0,0.836160,2.23,1.99,0.24,0,0.000000,0.0,0.24,4
603,262,7,1.86,2.13,0.0,0.00,0,0,0.977130,2.13,1.86,0.27,1,0.000000,0.0,0.27,0
131,271,2,1.86,2.18,0.0,0.06,0,0,0.944076,2.12,1.86,0.26,0,0.027523,0.0,0.32,2
821,228,4,1.79,1.79,0.0,0.00,0,0,0.740314,1.79,1.79,0.00,0,0.000000,0.0,0.00,4
945,258,2,1.86,2.18,0.0,0.00,0,0,0.018201,2.18,1.86,0.32,0,0.000000,0.0,0.32,2


In [27]:
linearSVM = svm.SVC(kernel= 'linear', C= 0.01)
linearSVM.fit(X_train, y_train)

SVC(C=0.01, kernel='linear')

##### b)

In [28]:
linearSVM.get_params()

{'C': 0.01,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [29]:
margin = 1 / np.sqrt(np.sum(linearSVM.coef_ ** 2))
margin

0.8689800260289876

In [30]:
len(linearSVM.support_vectors_)

622

In [36]:
linearSVM.support_vectors_[0]

array([2.72000e+02, 2.00000e+00, 1.86000e+00, 2.18000e+00, 0.00000e+00,
       6.00000e-02, 0.00000e+00, 0.00000e+00, 4.71378e-01, 2.12000e+00,
       1.86000e+00, 2.60000e-01, 0.00000e+00, 2.75230e-02, 0.00000e+00,
       3.20000e-01, 2.00000e+00])

In [31]:
testPreds = linearSVM.predict(X_test)
trainPreds = linearSVM.predict(X_train)

This classifier has 622 support vectors and a margin of 0.87, and an error on the test set of 0.23. This tells us that only 622 data points from the training set provide meaningful information for our model to create the separating hyperplane. We also see that because there is error both on the training and testing set that this classifier cannot cleanly separate the data using linear hyperplanes.

##### c)

In [32]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.20750000000000002    Test error is: 0.22962962962962963


In [17]:
def range_with_floats(start, stop, step):
    while stop > start:
        yield start
        start += step

In [20]:
# Set the parameters by cross-validation
tuned_parameters = [
    {"kernel": ["linear"], "C": list(range_with_floats(.5,10,.5))},
]

scores = ["accuracy"]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(max_iter = 1000000), tuned_parameters, scoring="accuracy")
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for accuracy



c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

Best parameters set found on development set:

{'C': 1.5, 'kernel': 'linear'}

Grid scores on development set:

0.828 (+/-0.044) for {'C': 0.5, 'kernel': 'linear'}
0.832 (+/-0.051) for {'C': 1.0, 'kernel': 'linear'}
0.835 (+/-0.058) for {'C': 1.5, 'kernel': 'linear'}
0.834 (+/-0.054) for {'C': 2.0, 'kernel': 'linear'}
0.835 (+/-0.048) for {'C': 2.5, 'kernel': 'linear'}
0.832 (+/-0.051) for {'C': 3.0, 'kernel': 'linear'}
0.832 (+/-0.044) for {'C': 3.5, 'kernel': 'linear'}
0.834 (+/-0.047) for {'C': 4.0, 'kernel': 'linear'}
0.824 (+/-0.055) for {'C': 4.5, 'kernel': 'linear'}
0.832 (+/-0.057) for {'C': 5.0, 'kernel': 'linear'}
0.834 (+/-0.053) for {'C': 5.5, 'kernel': 'linear'}
0.828 (+/-0.055) for {'C': 6.0, 'kernel': 'linear'}
0.832 (+/-0.052) for {'C': 6.5, 'kernel': 'linear'}
0.830 (+/-0.053) for {'C': 7.0, 'kernel': 'linear'}
0.765 (+/-0.242) for {'C': 7.5, 'kernel': 'linear'}
0.826 (+/-0.049) for {'C': 8.0, 'kernel': 'linear'}
0.828 (+/-0.048) for {'C': 8.5, 'kernel': 'linear'}
0.82

Using GridSearchCV as suggested in the homework slack, the best C value to use in this situation is C = 1.5. We do not necessarily check the one-stderr rule as we trust that GridSearchCV handles this in its calculations.

In [21]:
finalLinear = svm.SVC(kernel= 'linear', C= 1.5)
finalLinear.fit(X_train, y_train)

SVC(C=1.5, kernel='linear')

In [22]:
testPreds = finalLinear.predict(X_test)
trainPreds = finalLinear.predict(X_train)

##### e)

In [23]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.15874999999999995    Test error is: 0.16296296296296298


##### Part F
For a svc with a radial bassis function kernel, the gamma value is inversely proportional to the standard deviation. As gamma decreases, we can say that the standard deviation of our model is increasing. In this kernel, as gamma decreases, there is less of an emphasis on distance between points, making points that are farther away from each other to be considered as more similar. With a higher gamma, the model sees every point as more dissimilar, whicch drives the model to overfitting. When we have a smaller gamma, the model tends to generalize better and group more points similarly which results in us seeing a more simple model in that scenario.

#### G

In [49]:
kernelRBF = svm.SVC(kernel= 'rbf')
kernelRBF.fit(X_train, y_train)

testPreds = kernelRBF.predict(X_test)
trainPreds = kernelRBF.predict(X_train)


In [50]:
len(kernelRBF.support_vectors_)

630

The base RBF kernel has 630 support vectors and a test error of 0.37. There are the most support vectors in this model (although not by much) which signifies that it has the most complex hyperplane construction. The high test error goes to show that this type of kernel is sensitive to hyperparameter tuning.

In [24]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.39625    Test error is: 0.37037037037037035


In [33]:
# Set the parameters by cross-validation
tuned_parameters = [
    {"kernel": ["rbf"], "gamma": list(range_with_floats(.01,5,.5)), "C": list(range_with_floats(1,5,.5))},
]

scores = ["accuracy"]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(max_iter = 1000000), tuned_parameters, scoring="accuracy")
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'C': 4.5, 'gamma': 0.01, 'kernel': 'rbf'}

Grid scores on development set:

0.666 (+/-0.037) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.736 (+/-0.062) for {'C': 1, 'gamma': 0.51, 'kernel': 'rbf'}
0.704 (+/-0.055) for {'C': 1, 'gamma': 1.01, 'kernel': 'rbf'}
0.695 (+/-0.058) for {'C': 1, 'gamma': 1.51, 'kernel': 'rbf'}
0.696 (+/-0.068) for {'C': 1, 'gamma': 2.01, 'kernel': 'rbf'}
0.694 (+/-0.080) for {'C': 1, 'gamma': 2.51, 'kernel': 'rbf'}
0.700 (+/-0.084) for {'C': 1, 'gamma': 3.01, 'kernel': 'rbf'}
0.701 (+/-0.087) for {'C': 1, 'gamma': 3.51, 'kernel': 'rbf'}
0.699 (+/-0.082) for {'C': 1, 'gamma': 4.01, 'kernel': 'rbf'}
0.699 (+/-0.087) for {'C': 1, 'gamma': 4.51, 'kernel': 'rbf'}
0.696 (+/-0.059) for {'C': 1.5, 'gamma': 0.01, 'kernel': 'rbf'}
0.733 (+/-0.060) for {'C': 1.5, 'gamma': 0.51, 'kernel': 'rbf'}
0.700 (+/-0.065) for {'C': 1.5, 'gamma': 1.01, 'kernel': 'rbf'}
0.694 (+/-0.075) for {'C

The selected parameters for this model are C=4.5 and gamma = 0.01. We use GridSearchCV which executes cross validation to find these parameters, as specified in the slack channel.

In [47]:
finalRBF = svm.SVC(kernel= 'rbf', gamma = 0.01,C=4.5)
finalRBF.fit(X_train, y_train)

testPreds = finalRBF.predict(X_test)
trainPreds = finalRBF.predict(X_train)

In [48]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.1825    Test error is: 0.20740740740740737


##### h)

In [41]:
polyKernel = svm.SVC(kernel= 'poly', degree = 2)
polyKernel.fit(X_train, y_train)

testPreds = polyKernel.predict(X_test)
trainPreds = polyKernel.predict(X_train)

In [44]:
print(len(polyKernel.support_vectors_))

626


The base polynomial kernel SVC with a degree of 2 has 626 support vectors, and a test error of 0.37. Compared to the linear kernel, this base model has just about the same number of support vectors but a much higher test error rate. This goes to show that non-linear kernels are much more sensitive to hyperparameter tuning.

In [59]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.39625    Test error is: 0.37037037037037035


In [52]:
# Set the parameters by cross-validation
tuned_parameters = [
    {"kernel": ["poly"],"degree": [2], "gamma": list(range_with_floats(.001,1,.05)), "C": list(range_with_floats(1,5,.5))},
]

scores = ["accuracy"]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(max_iter = 1000000), tuned_parameters, scoring="accuracy")
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for accuracy



c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

...

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consid

Best parameters set found on development set:

{'C': 4.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly'}

Grid scores on development set:

0.815 (+/-0.047) for {'C': 1, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly'}
0.795 (+/-0.112) for {'C': 1, 'degree': 2, 'gamma': 0.051000000000000004, 'kernel': 'poly'}
0.757 (+/-0.118) for {'C': 1, 'degree': 2, 'gamma': 0.101, 'kernel': 'poly'}
0.761 (+/-0.124) for {'C': 1, 'degree': 2, 'gamma': 0.15100000000000002, 'kernel': 'poly'}
0.751 (+/-0.163) for {'C': 1, 'degree': 2, 'gamma': 0.201, 'kernel': 'poly'}
0.745 (+/-0.291) for {'C': 1, 'degree': 2, 'gamma': 0.251, 'kernel': 'poly'}
0.790 (+/-0.105) for {'C': 1, 'degree': 2, 'gamma': 0.301, 'kernel': 'poly'}
0.756 (+/-0.108) for {'C': 1, 'degree': 2, 'gamma': 0.351, 'kernel': 'poly'}
0.799 (+/-0.056) for {'C': 1, 'degree': 2, 'gamma': 0.40099999999999997, 'kernel': 'poly'}
0.749 (+/-0.073) for {'C': 1, 'degree': 2, 'gamma': 0.45099999999999996, 'kernel': 'poly'}
0.800 (+/-0.058) for {'C': 1, 'deg

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In this scenario, our selected parameters are C=4 and gamma = 0.001. As with the previous two models, we use GridSearchCV to execute the desired cross validation and find these parameters.

In [53]:
finalPoly = svm.SVC(kernel= 'poly',degree = 2, gamma= 0.001, C=4)
finalPoly.fit(X_train, y_train)

testPreds = finalPoly.predict(X_test)
trainPreds = finalPoly.predict(X_train)

In [54]:
print("Train error is:",1-accuracy_score(y_train, trainPreds), "   Test error is:",1-accuracy_score(y_test,testPreds))

Train error is: 0.16749999999999998    Test error is: 0.1777777777777778


##### i)

Overall based on this data, the linear kernel approach seems to give the best results. While the polynomial degree 2 kernel performs slightly better on the the test data, the linear kernel is simpler and easier to understand. We also look towards the cross validation exercise while tweaking parameters and see tha the polynomial kernel model is incredibly volatile. This lends us to believe that as the data may evolve over time, our chosen value for gamma may not perform well. For the linear kernel model, there is much less volatility in error rate as we change the parameters, and what variability there is stays within a range that performs much better. Therefore, the linear kernel approach is the best model to choose for this data and going forward as we gather more data.